In [74]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import random


In [75]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)


In [76]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


In [77]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


In [78]:
def linreg(X, w, b):  #@save
    """线性回归模型"""
    return torch.matmul(X, w) + b

def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()



In [79]:
lr = 0.03
num_epochs = 30
batch_size = 10
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    # for X, y in data_iter(batch_size, features, labels):
    l = loss(net(features, w, b), labels)  # X和y的小批量损失
    # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
    # 并以此计算关于[w,b]的梯度
    l.sum().backward()
    sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')


epoch 1, loss 78.352867
epoch 2, loss 356.175995
epoch 3, loss 1623.370850
epoch 4, loss 7417.300293
epoch 5, loss 33969.054688
epoch 6, loss 155908.093750
epoch 7, loss 717038.125000
epoch 8, loss 3304050.750000
epoch 9, loss 15251993.000000
epoch 10, loss 70522808.000000
epoch 11, loss 326590912.000000
epoch 12, loss 1514611072.000000
epoch 13, loss 7033556480.000000
epoch 14, loss 32702509056.000000
epoch 15, loss 152222023680.000000
epoch 16, loss 709292720128.000000
epoch 17, loss 3308170248192.000000
epoch 18, loss 15442928205824.000000
epoch 19, loss 72147162628096.000000
epoch 20, loss 337307739619328.000000
epoch 21, loss 1578059434033152.000000
epoch 22, loss 7387277713997824.000000
epoch 23, loss 34600784817553408.000000
epoch 24, loss 162145976180539392.000000
epoch 25, loss 760195811094036480.000000
epoch 26, loss 3565539462383403008.000000
epoch 27, loss 16729696138239672320.000000
epoch 28, loss 78523426096692592640.000000
epoch 29, loss 368675818942500962304.000000
epoc

In [80]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([ 3.8674e+10, -1.2667e+10], grad_fn=<SubBackward0>)
b的估计误差: tensor([4.0298e+10], grad_fn=<RsubBackward1>)
